In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

In [ ]:
# Load the CSV file
df = pd.read_csv('memory_match_frustration_data_200_samples.csv')

# Custom Dataset Class
class MemoryMatchDataset(Dataset):
    def __init__(self, dataframe, sequence_length=10):
        self.sequence_length = sequence_length
        self.data = dataframe[['reaction_time (sec)', 'incorrect_attempts', 'repeated_incorrect', 
                               'negative_penalty', 'score']].values.astype(np.float32)

        # Simulated frustration levels (heuristic for demo)
        self.targets = ((dataframe['incorrect_attempts'] + dataframe['repeated_incorrect'] + 
                         dataframe['negative_penalty']) / (dataframe['move_number'] + 1)).values.astype(np.float32)

    def __len__(self):
        return len(self.data) - self.sequence_length

    def __getitem__(self, idx):
        x = self.data[idx:idx + self.sequence_length]
        y = self.targets[idx + self.sequence_length - 1]
        return torch.tensor(x), torch.tensor(y)

# GRU Model
class FrustrationGRU(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FrustrationGRU, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.gru(x)
        out = self.fc(out[:, -1, :])
        return out

# Hyperparameters
INPUT_SIZE = 5
HIDDEN_SIZE = 32
OUTPUT_SIZE = 1
EPOCHS = 100
BATCH_SIZE = 4
LEARNING_RATE = 0.001

# Prepare Dataset and DataLoader
dataset = MemoryMatchDataset(df)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# Model, Loss, Optimizer
model = FrustrationGRU(INPUT_SIZE, HIDDEN_SIZE, OUTPUT_SIZE)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Training Loop
for epoch in range(EPOCHS):
    total_loss = 0
    for X_batch, y_batch in dataloader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs.squeeze(), y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {total_loss/len(dataloader):.4f}")

# Example Inference (Predict Frustration)
with torch.no_grad():
    sample_seq = torch.tensor(dataset[0][0]).unsqueeze(0)  
    predicted_frustration = model(sample_seq)
    print(f"Predicted Frustration Level: {predicted_frustration.item():.4f}")


Epoch 1/100, Loss: 0.1056
Epoch 2/100, Loss: 0.0271
Epoch 3/100, Loss: 0.0177
Epoch 4/100, Loss: 0.0127
Epoch 5/100, Loss: 0.0093
Epoch 6/100, Loss: 0.0071
Epoch 7/100, Loss: 0.0057
Epoch 8/100, Loss: 0.0048
Epoch 9/100, Loss: 0.0041
Epoch 10/100, Loss: 0.0032
Epoch 11/100, Loss: 0.0025
Epoch 12/100, Loss: 0.0022
Epoch 13/100, Loss: 0.0015
Epoch 14/100, Loss: 0.0013
Epoch 15/100, Loss: 0.0012
Epoch 16/100, Loss: 0.0010
Epoch 17/100, Loss: 0.0009
Epoch 18/100, Loss: 0.0007
Epoch 19/100, Loss: 0.0006
Epoch 20/100, Loss: 0.0006
Epoch 21/100, Loss: 0.0005
Epoch 22/100, Loss: 0.0004
Epoch 23/100, Loss: 0.0004
Epoch 24/100, Loss: 0.0003
Epoch 25/100, Loss: 0.0002
Epoch 26/100, Loss: 0.0002
Epoch 27/100, Loss: 0.0002
Epoch 28/100, Loss: 0.0002
Epoch 29/100, Loss: 0.0001
Epoch 30/100, Loss: 0.0001
Epoch 31/100, Loss: 0.0001
Epoch 32/100, Loss: 0.0001
Epoch 33/100, Loss: 0.0001
Epoch 34/100, Loss: 0.0000
Epoch 35/100, Loss: 0.0000
Epoch 36/100, Loss: 0.0000
Epoch 37/100, Loss: 0.0000
Epoch 38/1

C:\Users\KIIT01\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:66: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
